In [1]:
from test import load_data_sparse
from SGD import *
from ALS import *
from surprise_models import *
from surprise.model_selection import train_test_split

In [2]:
def calculate_rmse(real_labels, predictions):
    """Calculate RMSE."""
    return np.linalg.norm(real_labels - predictions) / np.sqrt(len(real_labels))

In [3]:
data_name = "data/47b05e70-6076-44e8-96da-2530dc2187de_data_train.csv"
test_name = "data/sampleSubmission.csv"

Load trainset:

In [4]:
sp_ratings, pd_ratings = load_data_sparse(data_name, False)

Convert it into surprise

In [5]:
spr_data = pandas_to_surprise(pd_ratings)

Split into test and dataset

In [6]:
trainset, testset = train_test_split(spr_data, test_size=.1)

get labels

In [7]:
_,_,labels = get_testset_indices(testset)

In [8]:
baseline = surprise_baseline(trainset, testset)

Estimating biases using als...


In [9]:
calculate_rmse(labels, baseline)

1.0000270278064725

In [10]:
svd = surprise_SVD(trainset, testset)

In [11]:
calculate_rmse(labels, svd)

1.0020748904168548

In [12]:
so = surprise_slopeOne(trainset, testset)

In [13]:
calculate_rmse(labels, so)

1.000550844353506

In [14]:
bsknn = surprise_baselineKNN(trainset, testset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [15]:
calculate_rmse(labels, bsknn)

0.9883088986305951

In [16]:
num_features = 40  # K in the lecture notes
lambda_user = 0.1
lambda_film = 0.1
stop_criterion = 1e-4

In [17]:
als = ALS_CV(trainset, testset, num_features, lambda_user, lambda_film, stop_criterion)

learn the matrix factorization using ALS...
RMSE: 1.0885379361895906.
RMSE: 1.0446836227982086.
RMSE: 1.0071044079361162.
RMSE: 0.9772986419708037.
RMSE: 0.9575377113888262.
RMSE: 0.9437770101915395.
RMSE: 0.9339080895372597.
RMSE: 0.9267495812519011.
RMSE: 0.9214784013817836.
RMSE: 0.9175172063134795.
RMSE: 0.9144782095587309.
RMSE: 0.9121018932720457.
RMSE: 0.9102113499318878.
RMSE: 0.9086834796559254.
RMSE: 0.9074311396743088.
RMSE: 0.9063917054514165.
RMSE: 0.9055194920304458.
RMSE: 0.9047806363818766.
RMSE: 0.9041496128014617.
RMSE: 0.903606850916588.
RMSE: 0.9031371004227797.
RMSE: 0.902728299887454.
RMSE: 0.9023707846156773.
RMSE: 0.9020567227414422.
RMSE: 0.9017797060771185.
RMSE: 0.9015344474052494.
RMSE: 0.9013165523958363.
RMSE: 0.901122344963561.
RMSE: 0.9009487316926147.
RMSE: 0.9007930953584756.
RMSE: 0.9006532104698802.
RMSE: 0.9005271757025561.
RMSE: 0.9004133594432011.
RMSE: 0.9003103556101321.
RMSE: 0.9002169476019075.
Iteration stopped, as iteration criterion 0.0001 

In [18]:
calculate_rmse(labels, als)

0.9878658179585607

In [19]:
sgd = matrix_factorization_SGD_CV(trainset, testset, num_features, lambda_user, lambda_film, stop_criterion)

learn the matrix factorization using SGD...
RMSE: 1.0411647662790124.
RMSE: 1.0180645192189566.
RMSE: 1.0094384123527471.
RMSE: 1.0065622174302191.
RMSE: 1.004430908125851.
RMSE: 1.0036462616829969.
RMSE: 1.0033836063118458.
RMSE: 1.0032539740132023.
RMSE: 1.0032227665707458.


In [20]:
calculate_rmse(labels, sgd)

1.0259306285053953

In [27]:
def surprise_SVDpp(trainset, finalset):

    options = {'method': 'als',
               'reg_i': 1.e-5,
               'reg_u': 14.6,
               'n_epochs': 10
               }

    algo = spr.SVDpp(n_factors=40, n_epochs=20, lr_all=0.001)

    algo.fit(trainset)
    predictions_final = algo.test(finalset)

    return spr_estimate_to_vect(predictions_final)

In [28]:
svdpp = surprise_SVDpp(trainset, testset)

In [29]:
calculate_rmse(labels, svdpp)

1.0017129237247184

# J'ai pas run SVDpp parce que ça prend du temps

In [31]:
X = [baseline, svd, so, bsknn, als, sgd, svdpp]
X = pd.DataFrame(X)
X = np.transpose(X)

from sklearn.linear_model import Ridge
linreg = Ridge(alpha=0.1, fit_intercept=False)
linreg.fit(X, labels)
linreg.coef_

#result = svd_final * linreg.coef_[0] + so_final * linreg.coef_[1] + als_final * linreg.coef_[2] + sgd_final * linreg.coef_[3]

array([-0.33202676, -0.24487176, -0.03659188,  0.29858978,  0.98595659,
        0.05355454,  0.30458821])

array([-0.09642474, -0.02721603, -0.19674828,  0.30887192,  1.0070824 ,
        0.03200952])